## Import Packages

In [42]:
import tensorflow as tf
import pandas as pd
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


## Read File

In [43]:
data = pd.read_csv('OnlineNewsPopularity.csv')
data.columns = [col.strip() for col in data.columns]


## Create isolation forest model to remove outliers
[API Link to IF](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest)

In [44]:
old_data_length = len(data)
IF_model = IsolationForest(contamination=0.1) # Remove 10% data
outliers = IF_model.fit_predict(data.drop(columns=['url', 'timedelta', 'shares'])) # Delete columns than don't need in IF and train IF model
data['outliers'] = outliers

# Remove outliers
data_outliers = data[data['outliers'] == -1]
new_data = data[data['outliers'] != -1].drop(columns = ['outliers'])

new_data_len = len(new_data)

print(f"There are total {old_data_length - new_data_len} rows been removied.")


/Users/will./anaconda3/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


There are total 3965 rows been removied.


## Creat a function to classify an article to different categories
* Great Article: Shares >= 90%
* Good Article: 70% <= Shares < 90%
* Normal Article: 30% <= Shares < 70%
* Bad article: Shares < 30%

In [45]:
def article_classifier(shares, top_percent):
    if shares >= top_percent[0.9]:
        return 'Great'
    elif shares >= top_percent[0.7]:
        return 'Good'
    elif shares >= top_percent[0.3]:
        return 'Normal'
    else:
        return 'Bad'
        

## Data Classifier

In [46]:
top_percent = new_data['shares'].quantile([0.3, 0.7, 0.9])

quality = []
for index, row in new_data.iterrows():
    quality.append(article_classifier(row['shares'], top_percent))

new_data['quality'] = quality


## Split dataset into x and y and one-hot encode quality column

In [47]:
x = new_data.drop(columns=['quality', 'url', 'timedelta', 'shares'], axis=1)
y = OneHotEncoder().fit_transform(new_data[['quality']]).toarray()


## Train test split

In [48]:

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=10)


## Scale the data to Improve the speed of training Model

In [49]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
scale_xTrain = standard_scaler.fit_transform(xTrain)
scale_xTest = standard_scaler.transform(xTest)

## Create and train model(NN)

In [50]:


def build_nn_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

mirror_strategy = tf.distribute.MirroredStrategy()
with mirror_strategy.scope():
    nn_model1 = build_nn_model(scale_xTrain.shape[1:], yTrain.shape[1])
    nn_model2 = build_nn_model(scale_xTrain.shape[1:], yTrain.shape[1])

    fit_speed = tf.data.Dataset.from_tensor_slices((scale_xTrain, yTrain))
    fit_speed = fit_speed.repeat().batch(100)

    nn_model1.fit(scale_xTrain, yTrain, epochs=10)
    nn_model2.fit(scale_xTrain, yTrain, epochs=10)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/10
 95/892 [==>...........................] - ETA: 0s - loss: 1.2675 - accuracy: 0.4000  

2023-08-31 13:55:32.340648: W tensorflow/core/framework/dataset.cc:956] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


892/892 [==============================] - 1s 556us/step - loss: 1.2097 - accuracy: 0.4363
Epoch 2/10
892/892 [==============================] - 0s 551us/step - loss: 1.1727 - accuracy: 0.4568
Epoch 3/10
892/892 [==============================] - 0s 560us/step - loss: 1.1585 - accuracy: 0.4679
Epoch 4/10
892/892 [==============================] - 1s 569us/step - loss: 1.1464 - accuracy: 0.4762
Epoch 5/10
892/892 [==============================] - 1s 605us/step - loss: 1.1349 - accuracy: 0.4818
Epoch 6/10
892/892 [==============================] - 1s 634us/step - loss: 1.1223 - accuracy: 0.4912
Epoch 7/10
892/892 [==============================] - 1s 604us/step - loss: 1.1101 - accuracy: 0.4972
Epoch 8/10
892/892 [==============================] - 1s 669us/step - loss: 1.0983 - accuracy: 0.5001
Epoch 9/10
892/892 [==============================] - 1s 598us/step - loss: 1.0837 - accuracy: 0.5115
Epoch 10/10
892/892 [==============================] - 1s 606us/step - loss: 1.0692 - accurac

## Create and train model(Random Forest)

In [51]:
RF_model = RandomForestClassifier(n_estimators=100)
RF_model.fit(scale_xTrain, yTrain.argmax(axis=1))


RandomForestClassifier()

## Make predictions with each model individually

In [52]:
nn1_pred = nn_model1.predict(scale_xTest)
nn2_pred = nn_model2.predict(scale_xTest)
RF_pred = tf.keras.utils.to_categorical(RF_model.predict(scale_xTest), num_classes=yTrain.shape[1])

223/223 [==============================] - 0s 293us/step


## Create ensemble model

In [53]:
ensemble_pred = (nn1_pred + nn2_pred + RF_pred) / 3
ensemble_labels = tf.argmax(ensemble_pred, axis=1).numpy()

## Model Evaluation

In [54]:
print(classification_report(yTest.argmax(axis=1), ensemble_labels))


              precision    recall  f1-score   support

           0       0.51      0.48      0.49      2087
           1       0.40      0.15      0.22      1444
           2       0.25      0.04      0.07       730
           3       0.44      0.69      0.54      2875

    accuracy                           0.45      7136
   macro avg       0.40      0.34      0.33      7136
weighted avg       0.43      0.45      0.41      7136

